In [34]:
import cv2
import numpy as np
from pathlib import Path
import sys; sys.path.append('..')
from scareddataset import iotools
from scareddataset import data_maniputlation as dm 
from scareddataset import calibrator as cb
from scareddataset import evaluation as evaluation

In [35]:
root_dataset_dir = Path('/home/dimitrisps/Datasets/Scared_stereo/scared')
sample_dir = Path('/home/dimitrisps/Datasets/Scared_stereo/scared/dataset6/keyframe5')
# image_3d = sample_dir / 'left_depth_map.tiff'
init_calib = sample_dir / 'endoscope_calibration.yaml'


In [36]:
calibrator = cb.StereoCalibrator()
# calib = calibrator.load(str(init_calib))
calib = calibrator.load('/home/dimitrisps/Documents/programming/SCARED_dataset_toolkit/complete_calib.json')

In [37]:
np.linalg.det(calib['R'])

1.0

In [38]:
left0 = cv2.imread(str(sample_dir / 'Left_Image.png'))
right0 = cv2.imread(str(sample_dir / 'Right_Image.png'))
gt0 = iotools.load_depthmap_xyz(str(sample_dir / 'left_depth_map.tiff'))
size = left0.shape[:2]

In [39]:
left_rect_m1, right_rect_m1 = calibrator.rectify(left0, right0)
calib_m1 = calibrator.calib.copy()
gt_m1 = gt0.copy()
gt_m1_rot = dm.transform_pts(gt_m1.reshape(-1,3),dm.create_RT(R = calib_m1['R1']))

disparity_img_m1 = dm.ptd3d_to_disparity(gt_m1_rot, calib_m1['P1'], calib_m1['P2'], size)


In [40]:
left_rect_0, right_rect_0 = calibrator.rectify(left0, right0, 0)
calib_0 = calibrator.calib.copy()
gt_0 = gt0.copy()
gt_0_rot = dm.transform_pts(gt_0.reshape(-1,3),dm.create_RT(R = calib_0['R1']))
disparity_img_0 = dm.ptd3d_to_disparity(gt_0_rot, calib_0['P1'], calib_0['P2'], size)

In [8]:
cv2.imwrite('./disparity_0.png', disparity_img_0.astype(np.uint8))
cv2.imwrite('./disparity_m1.png', disparity_img_m1.astype(np.uint8))

True

In [33]:
calibrator.save('test_calib.json')

In [8]:
pts3d_recon_0 = dm.disparity_to_original_scared(disparity_img_0.astype(np.float32), calib_0)
pts3d_recon_m1 = dm.disparity_to_original_scared(disparity_img_m1.astype(np.float32), calib_m1)

In [33]:
evaluation.xyz_error(gt0, pts3d_recon_0)

NameError: name 'pts3d_recon_0' is not defined

In [10]:
evaluation.xyz_error(gt0, pts3d_recon_m1)

(0.564536277812165, 0.8845962524414063, 0.7165283203125)

In [13]:
from scipy.interpolate import griddata


a = disparity_img_0.copy()
a[a==0]=np.nan


x, y = np.indices(a.shape)
interp_0 = np.array(a)
interp_0[np.isnan(interp_0)] = griddata(
    (x[~np.isnan(a)], y[~np.isnan(a)]), # points we know
    a[~np.isnan(a)],                    # values we know
    (x[np.isnan(a)], y[np.isnan(a)]))   # points to interpolate


In [14]:
a = disparity_img_m1.copy()
a[a==0]=np.nan


x, y = np.indices(a.shape)
interp_m1 = np.array(a)
interp_m1[np.isnan(interp_m1)] = griddata(
    (x[~np.isnan(a)], y[~np.isnan(a)]), # points we know
    a[~np.isnan(a)],                    # values we know
    (x[np.isnan(a)], y[np.isnan(a)]))   # points to interpolate

In [20]:
depthmap = dm.pts3d_to_depthmap(gt_0)
cv2.imwrite('depthmap.png', depthmap)

True

In [22]:
a = depthmap.copy()
a[a==0]=np.nan


x, y = np.indices(a.shape)
interp_depthmap = np.array(a)
interp_depthmap[np.isnan(interp_depthmap)] = griddata(
    (x[~np.isnan(a)], y[~np.isnan(a)]), # points we know
    a[~np.isnan(a)],                    # values we know
    (x[np.isnan(a)], y[np.isnan(a)]))   # points to interpolate
cv2.imwrite('./interpolated_depthmap.png',interp_depthmap)

True

In [15]:
cv2.imwrite('interpolated_0.png', interp_0.astype(np.uint8))
cv2.imwrite('interpolated_m1.png', interp_m1.astype(np.uint8))

True

In [18]:
interp_filt = interp_0.copy()
# interp_filt = cv2.GaussianBlur(interp_filt.astype(np.float32), (9,9),0.5)
# interp_filt = cv2.bilateralFilter(interp_filt.astype(np.float32),9,75,75)

interp_filt = cv2.GaussianBlur(interp_filt.astype(np.float32), (25,25),1, borderType=cv2.BORDER_REFLECT)
interp_filt = cv2.medianBlur(interp_filt.astype(np.float32), 5)
cv2.imwrite('interpolated_filt_0.png', interp_filt.astype(np.uint8))

True

In [32]:
pts3d_recon_0_gb = dm.disparity_to_original_scared(interp_filt.astype(np.float32), calib_0)
evaluation.xyz_error(gt0, pts3d_recon_0_gb)


(0.8778428810334604, 0.8845962524414063, 0.6964210510253906)

In [20]:
iotools.export_ply('./ds1_kf1.ply',pts3d_recon_0_gb.reshape(-1,3))

In [79]:
disparity_img_0_gb = cv2.medianBlur(disparity_img_0.astype(np.float32), 1)
cv2.imwrite('./median.png',disparity_img_0_gb.astype(np.uint8))

True

In [80]:
disparity_img_0_gb = cv2.GaussianBlur(disparity_img_0.astype(np.float32), (5,5),0)
cv2.imwrite('./gaussian.png',disparity_img_0_gb.astype(np.uint8))

True

In [81]:
disparity_img_0_gb = cv2.bilateralFilter(disparity_img_0.astype(np.float32),9,75,75)
cv2.imwrite('./bilateral.png',disparity_img_0_gb.astype(np.uint8))

True

In [78]:
pts3d_recon_0_gb = dm.disparity_to_original_scared(disparity_img_0_gb.astype(np.float32), calib_0)
evaluation.xyz_error(gt0, pts3d_recon_0_gb)

(85.2355257186807, 0.5900093078613281, 0.47687454223632814)

In [100]:
scared_depthmap = iotools.load_depthmap_xyz(str(image_3d))
scene_pts_3d = scared_depthmap.reshape(-1,3) #xyz

pixel_locs = cv2.projectPoints(scene_pts_3d, np.eye(3), np.zeros(3), calib['K1'], calib['D1'])[0].squeeze()

In [112]:
pixel_locs = cv2.projectPoints(scene_pts_3d, np.eye(3), np.zeros(3), calib['K1'], np.zeros(5))[0].squeeze()

In [158]:
pixel_locs.shape

(1310720, 2)

In [159]:
pixel_locs = pixel_locs.reshape(1024,1280,2)

In [160]:
img_loc = np.mgrid[:1024,:1280].transpose(1,2,0)[:,:,::-1]

In [161]:
results = np.sqrt(np.sum((pixel_locs - img_loc)**2, axis=2))


In [162]:
cv2.imwrite('./diff_no_dist.png', (results*50).astype(np.uint8))

True

In [163]:
print(np.nanmean(results.reshape(-1)))

1.5787817042295778


In [124]:
video = cv2.VideoCapture(str(Path('/home/dimitrisps/Datasets/Scared_stereo/keyframe_1/data')/ 'rgb.mp4'))
left_dir = (Path('/home/dimitrisps/Datasets/Scared_stereo/keyframe_1/data') /'left')
right_dir = (Path('/home/dimitrisps/Datasets/Scared_stereo/keyframe_1/data') /'right')
left_dir.mkdir(exist_ok=True)
right_dir.mkdir(exist_ok=True)
i=0
while(video.isOpened()):
    ret, frame = video.read()
    if not ret:
        break
    left = frame[:1024]
    right= frame[1024:]
    
    cv2.imwrite(str(left_dir / ('{:06d}.png'.format(i))), left)
    cv2.imwrite(str(right_dir / ('{:06d}.png'.format(i))), right)
    i+=1
video.release()
    # calibrator.save(str(root_dir/'stereo_calib.json'))